# MarkdownHeaderTextSplitter

Hướng dẫn này giới thiệu cách tách **tài liệu Markdown** hiệu quả bằng `MarkdownHeaderTextSplitter` của LangChain. Công cụ này chia tài liệu thành các phần có ý nghĩa dựa trên tiêu đề Markdown, giữ nguyên cấu trúc tài liệu để xử lý nội dung một cách hệ thống.

Ngữ cảnh và cấu trúc của tài liệu rất quan trọng để embedding văn bản hiệu quả. Việc chỉ đơn giản chia văn bản là không đủ; việc duy trì các kết nối ngữ nghĩa là chìa khóa để tạo ra các biểu diễn vector toàn diện hơn. Điều này đặc biệt đúng khi xử lý các tài liệu lớn, vì việc giữ nguyên ngữ cảnh có thể tăng cường đáng kể độ chính xác của các hoạt động phân tích và tìm kiếm tiếp theo.

`MarkdownHeaderTextSplitter` chia tài liệu theo các bộ tiêu đề được chỉ định, quản lý nội dung dưới mỗi nhóm tiêu đề thành các chunk riêng biệt. Điều này cho phép xử lý nội dung hiệu quả trong khi vẫn duy trì tính mạch lạc cấu trúc của tài liệu.


## Basic Usage of MarkdownHeaderTextSplitter

`MarkdownHeaderTextSplitter` tách văn bản định dạng Markdown dựa trên tiêu đề. Đây là cách sử dụng nó:

-   Đầu tiên, bộ tách chia văn bản dựa trên các tiêu đề Markdown tiêu chuẩn (#, ##, ###, v.v.).
-   Lưu trữ Markdown bạn muốn tách trong một biến có tên là `markdown_document`.
-   Bạn sẽ cần một danh sách có tên là `headers_to_split_on`. Danh sách này sử dụng tuples để xác định các cấp tiêu đề bạn muốn tách và những gì bạn muốn gọi chúng.
-   Bây giờ, hãy tạo một đối tượng `markdown_splitter` bằng lớp `MarkdownHeaderTextSplitter` và cung cấp cho nó danh sách `headers_to_split_on` đó.
-   Để thực sự tách văn bản, hãy gọi phương thức `split_text` trên đối tượng `markdown_splitter` của bạn, chuyển vào `markdown_document` của bạn.


In [1]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

# Define a markdown document as a string
markdown_document = "# Title\n\n## 1. SubTitle\n\nHi this is Jim\n\nHi this is Joe\n\n### 1-1. Sub-SubTitle \n\nHi this is Lance \n\n## 2. Baz\n\nHi this is Molly"
print(markdown_document)

# Title

## 1. SubTitle

Hi this is Jim

Hi this is Joe

### 1-1. Sub-SubTitle 

Hi this is Lance 

## 2. Baz

Hi this is Molly


In [2]:
headers_to_split_on = [  # Define header levels and their names for document splitting
    (
        "#",
        "Header 1",
    ),  # Header level 1 is marked with '#' and named 'Header 1'
    (
        "##",
        "Header 2",
    ),  # Header level 2 is marked with '##' and named 'Header 2'
    (
        "###",
        "Header 3",
    ),  # Header level 3 is marked with '###' and named 'Header 3'
]

# Create a MarkdownHeaderTextSplitter object to split text based on markdown headers
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
# Split markdown_document by headers and store in md_header_splits
md_header_splits = markdown_splitter.split_text(markdown_document)
# Print the split results
for header in md_header_splits:
    print(f"{header.page_content}")
    print(f"{header.metadata}", end="\n=====================\n")

Hi this is Jim  
Hi this is Joe
{'Header 1': 'Title', 'Header 2': '1. SubTitle'}
Hi this is Lance
{'Header 1': 'Title', 'Header 2': '1. SubTitle', 'Header 3': '1-1. Sub-SubTitle'}
Hi this is Molly
{'Header 1': 'Title', 'Header 2': '2. Baz'}


### Header Retention in Split Output

Theo mặc định, `MarkdownHeaderTextSplitter` loại bỏ tiêu đề khỏi các chunk đầu ra.

Tuy nhiên, bạn có thể định cấu hình bộ tách để giữ lại các tiêu đề này bằng cách đặt tham số `strip_headers` thành `False`.

Ví dụ:


In [3]:
markdown_splitter = MarkdownHeaderTextSplitter(
    # Specify headers to split on
    headers_to_split_on=headers_to_split_on,
    # Set to keep headers in the output
    strip_headers=False,
)
# Split markdown document based on headers
md_header_splits = markdown_splitter.split_text(markdown_document)
# Print the split results
for header in md_header_splits:
    print(f"{header.page_content}")
    print(f"{header.metadata}", end="\n=====================\n")

# Title  
## 1. SubTitle  
Hi this is Jim  
Hi this is Joe
{'Header 1': 'Title', 'Header 2': '1. SubTitle'}
### 1-1. Sub-SubTitle  
Hi this is Lance
{'Header 1': 'Title', 'Header 2': '1. SubTitle', 'Header 3': '1-1. Sub-SubTitle'}
## 2. Baz  
Hi this is Molly
{'Header 1': 'Title', 'Header 2': '2. Baz'}


## Combining with Other Text Splitters

Sau khi tách theo tiêu đề Markdown, bạn có thể xử lý thêm nội dung trong mỗi nhóm Markdown bằng bất kỳ bộ tách văn bản mong muốn nào.

Trong ví dụ này, chúng ta sẽ sử dụng `RecursiveCharacterTextSplitter` để minh họa cách kết hợp hiệu quả các phương pháp tách khác nhau.


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

markdown_document = "# Intro \n\n## History \n\nMarkdown[9] is a lightweight markup language for creating formatted text using a plain-text editor. John Gruber created Markdown in 2004 as a markup language that is appealing to human readers in its source code form.[9] \n\nMarkdown is widely used in blogging, instant messaging, online forums, collaborative software, documentation pages, and readme files. \n\n## Rise and divergence \n\nAs Markdown popularity grew rapidly, many Markdown implementations appeared, driven mostly by the need for \n\nadditional features such as tables, footnotes, definition lists,[note 1] and Markdown inside HTML blocks. \n\n#### Standardization \n\nFrom 2012, a group of people, including Jeff Atwood and John MacFarlane, launched what Atwood characterised as a standardisation effort. \n\n# Implementations \n\nImplementations of Markdown are available for over a dozen programming languages."
print(markdown_document)

# Intro 

## History 

Markdown[9] is a lightweight markup language for creating formatted text using a plain-text editor. John Gruber created Markdown in 2004 as a markup language that is appealing to human readers in its source code form.[9] 

Markdown is widely used in blogging, instant messaging, online forums, collaborative software, documentation pages, and readme files. 

## Rise and divergence 

As Markdown popularity grew rapidly, many Markdown implementations appeared, driven mostly by the need for 

additional features such as tables, footnotes, definition lists,[note 1] and Markdown inside HTML blocks. 

#### Standardization 

From 2012, a group of people, including Jeff Atwood and John MacFarlane, launched what Atwood characterised as a standardisation effort. 

# Implementations 

Implementations of Markdown are available for over a dozen programming languages.


First, use `MarkdownHeaderTextSplitter` to split the Markdown document based on its headers.

In [5]:
headers_to_split_on = [
    ("#", "Header 1"),  # Specify the header level and its name to split on
    # ("##", "Header 2"),
]

# Split the markdown document based on header levels
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, strip_headers=False
)
md_header_splits = markdown_splitter.split_text(markdown_document)
# Print the split results
for header in md_header_splits:
    print(f"{header.page_content}")
    print(f"{header.metadata}", end="\n=====================\n")

# Intro  
## History  
Markdown[9] is a lightweight markup language for creating formatted text using a plain-text editor. John Gruber created Markdown in 2004 as a markup language that is appealing to human readers in its source code form.[9]  
Markdown is widely used in blogging, instant messaging, online forums, collaborative software, documentation pages, and readme files.  
## Rise and divergence  
As Markdown popularity grew rapidly, many Markdown implementations appeared, driven mostly by the need for  
additional features such as tables, footnotes, definition lists,[note 1] and Markdown inside HTML blocks.  
#### Standardization  
From 2012, a group of people, including Jeff Atwood and John MacFarlane, launched what Atwood characterised as a standardisation effort.
{'Header 1': 'Intro'}
# Implementations  
Implementations of Markdown are available for over a dozen programming languages.
{'Header 1': 'Implementations'}


Now, we'll further split the output of the `MarkdownHeaderTextSplitter` using the `RecursiveCharacterTextSplitter`.

In [6]:
chunk_size = 200  # Specify the size of each split chunk
chunk_overlap = 20  # Specify the number of overlapping characters between chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# Split the document into chunks by characters
splits = text_splitter.split_documents(md_header_splits)
# Print the split results
for header in splits:
    print(f"{header.page_content}")
    print(f"{header.metadata}", end="\n=====================\n")

# Intro  
## History
{'Header 1': 'Intro'}
Markdown[9] is a lightweight markup language for creating formatted text using a plain-text editor. John Gruber created Markdown in 2004 as a markup language that is appealing to human readers in its
{'Header 1': 'Intro'}
readers in its source code form.[9]
{'Header 1': 'Intro'}
Markdown is widely used in blogging, instant messaging, online forums, collaborative software, documentation pages, and readme files.  
## Rise and divergence
{'Header 1': 'Intro'}
As Markdown popularity grew rapidly, many Markdown implementations appeared, driven mostly by the need for
{'Header 1': 'Intro'}
additional features such as tables, footnotes, definition lists,[note 1] and Markdown inside HTML blocks.  
#### Standardization
{'Header 1': 'Intro'}
From 2012, a group of people, including Jeff Atwood and John MacFarlane, launched what Atwood characterised as a standardisation effort.
{'Header 1': 'Intro'}
# Implementations  
Implementations of Markdown are avail